**1. Data Preparation**

In [ ]:
#Importing necessary libraries
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#Loading the dataset to the environment
#dataset = pd.read_csv('C:\\Users\\Laptop Outlet\\Downloads\\academic_dataset.csv')
dataset = pd.read_csv('academic_dataset.csv')

In [ ]:
#Checking if there are any missing values
missing_vals = dataset.isnull().sum()

In [ ]:
#Handling missing values by replacing with the first mode
dataset.fillna(dataset.mode().iloc[0], inplace=True)

#Handling missing values by replacing with forward-filling (last valid value before null)
#dataset.fillna(method='ffill', inplace=True)

In [ ]:
#Encoding categorical variables with LabelEncoder()
dataset_columns = ['gender', 'NationalITy', 'PlaceofBirth', 'StageID', 'GradeID', 'SectionID', 'Topic', 'Semester', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'StudentAbsenceDays']

label = LabelEncoder()
for column in dataset_columns:
  dataset[column] = label.fit_transform(dataset[column])

In [ ]:
#Splitting other features and target variables
x = dataset.drop(columns = ['Class'])
y = dataset['Class']

In [ ]:
#Encoding the target variable after separation
label_class = LabelEncoder()
y = label_class.fit_transform(y)

In [ ]:
#Scaling numerical features using StandardScaler
standard_scaler = StandardScaler()
x[['raisedhands', 'VisITedResources', 'AnnouncementsView', 'Discussion']] = standard_scaler.fit_transform(x[['raisedhands', 'VisITedResources', 'AnnouncementsView','Discussion']])

In [ ]:
#Splitting the dataset as training set and validation set
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

**2. Model Design**

In [ ]:
#Creating a deep learning model
model = models.Sequential()

In [ ]:
#Input layer
model.add(layers.InputLayer(input_shape=(x_train.shape[1],)))

In [ ]:
#Adding the first hidden layer
model.add(layers.Dense(32, activation='relu'))

#Using Dropout layer to avoid overfitting
model.add(layers.Dropout(0.5))

#Adding the second hidden layer
model.add(layers.Dense(64, activation='relu'))

In [ ]:
#Output layer for classification - using softmax function
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Displaying the model summary
model.summary()

**3. Training**

In [ ]:
#Implementing early stopping to avoid overfitting
es = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
#Experimenting with different hyperparameters
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Training the model
result = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_val, y_val), callbacks=[es])

**4. Evaluation**

In [ ]:
#Evaluating the trained model with validation set
val_loss, v_accuracy = model.evaluate(x_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {v_accuracy}")

In [ ]:
#Making predictions
y_prediction = model.predict(x_val)
y_prediction_classes = y_prediction.argmax(axis=1)

In [ ]:
#Classification metrics (confusion matrix containing precision, recall, f1-score and support)
print("Classification Report:")
print(classification_report(y_val, y_prediction_classes))

In [ ]:
#Calculating the accuracy
accuracy = accuracy_score(y_val, y_prediction_classes)
print(f"Accuracy: {accuracy}")